# Youtube 인기 급상승 채널로부터 크롤링 진행

In [73]:
# scraping에 필요한 라이브러리 호출
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import re
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action = 'ignore')

## 최신 기준 상위 50개

In [80]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
url = "https://www.youtube.com/feed/trending"

options = webdriver.ChromeOptions()
# 크롬창 띄우기 X
# options.add_argument('headless')
options.add_argument('window-size=1920*1080')
# 크롤링 성능 향상
options.add_argument('disable-gpu')
options.add_argument('lang=ko_KR, en_US')
options.add_argument('user-agent=' + user_agent)

driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = options)
driver.get(url)
driver.implicitly_wait(3)

# 카테고리별 클릭 버튼 추가하기
# ~~~ code

# 페이지 스크롤 다운 (모든 동영상 로딩)
last_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(5)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    time.sleep(1.5)
    # 팝업 제거
    try:
        driver.find_element_by_css_selector('#dismiss-button > a').click()

    except:
        pass
    
# 모든 동영상 링크 수집
video_links = []
for video in driver.find_elements(By.XPATH, "//a[@id='video-title']"):
    video_links.append(video.get_attribute("href"))

    
# 상위 영상 n개 추출
video_links = video_links[:2]

In [82]:
html_sources = []
for video_link in video_links:
    driver.get(video_link)
    # continue
    # driver.maximize_window()
    # 댓글 로딩 대기
    driver.implicitly_wait(10)
    # 페이지 스크롤 다운 (모든 동영상 로딩) 하면 너무 오래걸림.. (문제 발생)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(2)
    # 팝업 제거
    try:
        driver.find_element_by_css_selector("#dismiss-button > a").click()
    except:
        pass
    
    time.sleep(2)
    
    html_source = driver.page_source
    html_sources.append(html_source)

In [83]:
# 각 url 별 html 코드 잘 append 되었나 checking
len(html_sources)

2

In [84]:
# 빈 데이터 프레임 생성
df = pd.DataFrame(columns = ['Youtube_ID', 'Comment'])
df

,Youtube_ID,Comment


In [85]:
for html_source in html_sources:
    # html 코드 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    # id 리스트
    id_list = soup.select('div#header-author > h3 > #author-text > span') 
    # comment 리스트  
    comment_list = soup.select('yt-formatted-string#content-text') 
    
    # 파싱해서 넣을 실제 데이터 리스트
    youtube_id = []
    youtube_comment = []
    # 각 comment 별 id, comment 출력
    for i in range(len(comment_list)):
        id_ = id_list[i].text
        id_ = id_.replace('\n', '').replace('\t', '').replace(' ', '').strip()
        youtube_id.append(id_) # 댓글 작성자
        
        comment_ = comment_list[i].text
        comment_ = comment_.replace('\n', '').replace('\t', '').replace('\r', '').strip()
        youtube_comment.append(comment_) # 댓글 내용

    youtube_data = {'Youtube_ID' : youtube_id, 'Comment' : youtube_comment}
    youtube_df = pd.DataFrame(youtube_data)
    df = pd.concat([df, youtube_df], axis=0)

In [91]:
df.reset_index(drop=True, inplace=True)

In [94]:
# 5/2 12시 48분 기준, 원래 댓글 50,000 개 -> 실제 읽어온 데이터 2665개. 

In [92]:
df

,Youtube_ID,Comment
0,@twaimz,"STUNNING, BEYOND TALENTED, BEYOND GORGEOUS"
1,@5.12ksubschallengewith0Video,Классная песня!!Девочки молодцыы!!!!Кто всегда...
2,@emelybonilla2532,10M¿11M?12M?13M?14M?15M?16M?17M?18M?19M?20M?ES...
3,@5.12ksubschallengewith0Video,"ДАВАЙТЕ ПОКАЖЕМ, НАСКОЛЬКО СИЛЬНО МЫ ЛЮБИМ И П..."
4,@albertsan8560,"VAMOS POR ESOS 10M DE VISTAS, GENTEEE!!! ESTAS..."
...,...,...
2660,@soul553,The song is so bad
2661,@1284-sc2yi,노잼
2662,@shahadalbalawi4559,Horrible
2663,@lisamylove14,"what kind of a song is this , so repetitive an..."


In [95]:
df.to_csv('youtube_crawling_sample.csv', index=False)

# Comment Visualization